<a href="https://colab.research.google.com/github/lehuong240823/svm-skin-disease-detector-anvil/blob/main/notebooks/Skin_Disease_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.5/124.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00


In [3]:
import anvil.server
import anvil.media
import os
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from sklearn.svm import SVC
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def preprocessing(image):
  norm_img = np.zeros((image.shape[0], image.shape[1]))
  print(norm_img.all)
  image = cv.normalize(image, norm_img, 0, 255, cv.NORM_MINMAX)
  image = cv.resize(image, (64, 128), interpolation = cv.INTER_NEAREST)
  image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
  image = cv.fastNlMeansDenoising(image, image, 3, 7, 21)
  return image

In [6]:
def segmentation(image):
  image = cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
  return image

In [7]:
def augmentation(image):
  height, width = image.shape[:2]
  center = (width/2, height/2)
  for i in range(0, 4):
    rotate_matrix = cv.getRotationMatrix2D(center=center, angle=90*i, scale=1)
    rotated_image = cv.warpAffine(src=image, M=rotate_matrix, dsize=(width, height))
    for j in range(-1, 2):
      aug_img = cv.flip(rotated_image, int(j))
      x.append(aug_img)
      y.append(class_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/public/SVM_Skin Disease/dataset'
classes = []
for _, dirs, _ in os.walk(data_dir):
  classes = sorted(dirs)
  break

In [9]:
x = []
y = []
for idx, class_name in enumerate(classes):
  class_path = os.path.join(data_dir, class_name)
  for _, _, files in os.walk(class_path + '/'):
    for file_name in files:
      img_path = class_path + '/' + file_name
      img = cv.imread(img_path)
      img = preprocessing(img)
      img = segmentation(img)
      x.append(img)
      y.append(class_name)
      augmentation(img)

<built-in method all of numpy.ndarray object at 0x7faa4c737ab0>
<built-in method all of numpy.ndarray object at 0x7faa4c1f1770>
<built-in method all of numpy.ndarray object at 0x7faa4c1f1dd0>
<built-in method all of numpy.ndarray object at 0x7faa4c1f21f0>
<built-in method all of numpy.ndarray object at 0x7faa4c1f26d0>
<built-in method all of numpy.ndarray object at 0x7faa4c1f2bb0>
<built-in method all of numpy.ndarray object at 0x7faa4c1f3090>
<built-in method all of numpy.ndarray object at 0x7faa4c1f3690>
<built-in method all of numpy.ndarray object at 0x7faa4c1f3b70>
<built-in method all of numpy.ndarray object at 0x7faa4c188090>
<built-in method all of numpy.ndarray object at 0x7faa4c188570>
<built-in method all of numpy.ndarray object at 0x7faa4c188a50>
<built-in method all of numpy.ndarray object at 0x7faa4c188f30>
<built-in method all of numpy.ndarray object at 0x7faa4c189410>
<built-in method all of numpy.ndarray object at 0x7faa4c1898f0>
<built-in method all of numpy.ndarray ob

In [ ]:
img_train, img_test, labels_train, labels_test = train_test_split(x, y, random_state = 42, test_size = 0.3, shuffle = True)

In [ ]:
def get_hog_features(img, orient = 8, pix_per_cell = 16, cell_per_block = 4, feature_vector = True):
  features = hog(img, orientations = orient,
                 pixels_per_cell = (pix_per_cell, pix_per_cell),
                 cells_per_block = (cell_per_block, cell_per_block),
                 transform_sqrt = True,
                 feature_vector = feature_vector)
  return features

In [ ]:
features_train = []
for img in img_train:
  features_hog = get_hog_features(img)
  features_train.append(features_hog)

features_test = []
for img in img_test:
  features_hog_test = get_hog_features(img)
  features_test.append(features_hog_test)

In [ ]:
x_hog_train = np.array(features_train)
y_train = np.array(labels_train)
x_hog_test = np.array(features_test)
y_test = np.array(labels_test)

In [ ]:
model_svm = SVC(kernel='rbf', C=400)
model_svm.fit(x_hog_train, y_train)

SVC(C=400)

In [ ]:
y_predict = model_svm.predict(x_hog_test)
print ('Độ chính xác: ', round(model_svm.score(x_hog_test, y_test)*100, 1), '%')

Độ chính xác:  75.2 %


In [ ]:
anvil.server.connect("server_LYJDWULBS3SQIIQEK3LK6C6N-E7TY7TEIZFQ4KGWK")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
def file_to_image(img_file):
  file_to_bytes = img_file.get_bytes()
  bytes_to_str = str(file_to_bytes)
  encoded_str = bytes_to_str.encode()
  decoded_str = encoded_str.decode('unicode-escape').encode('ISO-8859-1')[2:-1]
  bytes_str_to_arr = np.frombuffer(file_to_bytes, dtype=np.uint8)
  img = cv.imdecode(bytes_str_to_arr, cv.IMREAD_COLOR)
  return img

In [ ]:
@anvil.server.callable
def predict_desease(file):
  img = file_to_image(file)
  img = preprocessing(img)
  img = segmentation(img)
  img_feature = get_hog_features(img)
  img_array = np.array(img_feature)
  classification = model_svm.predict(img_array.reshape(1,-1))
  return classification[0]

In [ ]:
anvil.server.wait_forever()

<built-in method all of numpy.ndarray object at 0x7f3026077510>
<built-in method all of numpy.ndarray object at 0x7f30260775d0>
<built-in method all of numpy.ndarray object at 0x7f3026076790>
<built-in method all of numpy.ndarray object at 0x7f3026077030>
<built-in method all of numpy.ndarray object at 0x7f3026077810>
<built-in method all of numpy.ndarray object at 0x7f30260775d0>
<built-in method all of numpy.ndarray object at 0x7f3026077150>
<built-in method all of numpy.ndarray object at 0x7f3026077870>
<built-in method all of numpy.ndarray object at 0x7f3026077510>
<built-in method all of numpy.ndarray object at 0x7f3026077810>
<built-in method all of numpy.ndarray object at 0x7f30260777b0>
<built-in method all of numpy.ndarray object at 0x7f3026077ab0>
<built-in method all of numpy.ndarray object at 0x7f3026077c30>
<built-in method all of numpy.ndarray object at 0x7f30260777b0>
<built-in method all of numpy.ndarray object at 0x7f3026077a50>
<built-in method all of numpy.ndarray ob